# Lakeflow

## ¿Qué es Data Lakeflow y para qué sirve?

Data LakeFlow es una herramienta diseñada para orquestar, automatizar y gestionar flujos de datos dentro de un Data Lake. Permite definir, programar y monitorear procesos de ingestión, transformación y movimiento de datos de manera eficiente y escalable. Su objetivo principal es facilitar la integración de datos desde múltiples fuentes, asegurar la calidad y trazabilidad de los datos, y optimizar el procesamiento para análisis avanzado y toma de decisiones empresariales.

## Conceptos básicos

### Tablas Streaming
- Procesamiento incremental
- Exactamente una vez
- Usadas para la ingesta de datos
- Cargas solo de tipo append (una sola inserción)
- Transmisión de baja latencia

Ejemplo de uso

```sql
CREATE STREAMING TABLE web_clicks
AS 
SELECT * FROM STREAM read_files('/volume/path');
```
-----
```sql
CREATE STREAMING TABLE server_logs
AS 
SELECT from_json(...) data
FROM STREAM read_kafka(...)
```

### Vistas Materializadas
- Resultados de consultas almacenadas en caché con actualizaciones programadas
- Precalculo para un acceso rápido
- Puede ser incremental pero no necesariamente
- Usado para transformaciones complejas
- Consultas analíticas
- Resultados siempre correctos
- Acelera las consultas en los dashboards
- Mejora la frescura de los datos

Ejemplo de uso

```sql
CREATE MATERIALIZED VIEW customers_orders
AS 
SELECT 
 customers.name,
 sum(orders.amount),
 orders.orderdate
FROM orders
LEFT JOIN customers ON 
  orders.custkey = customers.c_custkey
GROUP BY 
name,
orderdate;
```

### Flujos
- Alimenta a tablas streaming y a vistas materializadas
- Admite semántica streaming o batch


## Data Quality

La calidad de los datos se basa en expectativas (expectations) los cuales:

- Definen la calidad de los datos y los controles de integridad dentro de la canalización con expectativas
- Aborda los errores de calidad con políticas flexibles como fail, drop, alert, quarantine
- Todas las ejecuciones de pipelines y métricas de calidad se capturan, rastrean e informan

Ejemplo de uso:

```sql
CREATE STREAMING TABLE fire_account_bronze 
AS 
( 
  CONSTRAINT valid_account_open_dt EXPECT (account_dt. is note null and (account_close_dt > account_open_dt)) ON VIOLATION DROP ROW
  COMMENT "Bronze table with valid account ids"
  SELECT * FROM fire_account_raw ...
)
```

## Development vs Production

Integración rápida 

### Development mode
- Reutiliza un clúster para una iteración rápida
- Sin re-intentos en caso de error, lo que permite una interación más rápida
- Para ejecuciones activadas desde el editor o la interfaz de creación

### Production mode

- Reduce los costos apagando los clústers en cuanto terminan (menos de 5 minutos)
- Usa re-intentos escalonados inclidos los reinicios de clústers y garantizan la fiabilidad ante problemas transitorios
- Detiene los pipelines programados

## Modulariza tu código
Evida codificar de forma rígida las rutas, nombres, catálogos o esquemas

La configuración de un pipeline es un mala de **key-value pais** que puede utilizarse para parametrizar el código. Esto permite:

- Mejorar la legibilidad del codigo
- Reutilizar el codigo en varias cadenas

Ejemplo de uso:

- Ve a la opción de configuración del pipeline y crea una variable "volumen" con una ruta asignada /path
- En el código puedes leer la variable asi:

SQL:

```sql
CREATE STREAMING TABLE  data AS
SELECT * FROM read_files("${volumen}","json")
```

Python:

```python
@dp.table
def data():
  input_path = spark.conf.get("volumen")
  spark.readStream.format("cloudFiles").load(input_path)
``` 

## Observabilidad

Contiene toda la información relacionada con el pipeline, tablas o control de datos

```sql
select *
from event_log(table( catalog.schema.table_name ));

select * from event_log('pipeline-id');
```
